In [2]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [14]:
mu = 1
sigma = 2.0
learning_rate = 1e-3
epsilon = 1.0
epochs = 1000
batch_size = 2

In [15]:
# define linear model
model = nn.Linear(batch_size, batch_size, bias=False)

# model parameters
# print(list(model.parameters()))

# define loss
def linear_loss(output, target):
    return output @ target

# define optimizer
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)

# FGSM attack
def fgsm(model, X, y, epsilon):
    """ Construct FGSM adversarial examples on the examples X"""
    delta = torch.zeros_like(X, requires_grad=True)
    loss = linear_loss(model(X + delta), y)
    loss.backward()
    return epsilon * delta.grad.detach().sign()

def epoch_adversarial(loader, model):
    total_loss = 0.
    for X, y in loader:
        delta = fgsm(model, X, y, epsilon)
        # perturbed training data
        x_pert = X + delta # .float()
        # predicted output
        y_pred = model(x_pert)
        # calculate linear loss
#         print("x_pert", x_pert)
#         print("y_pred", y_pred)
        loss = linear_loss(y_pred, y)

        # clear gradients wrt to parameters
        opt.zero_grad()
        # get gradients wrt to parameters
        loss.backward()
        opt.step()
        
#         total_err += (y_pred.max(dim=1) != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_loss / len(loader.dataset)

def epoch_test(loader, model):
    total_loss = 0.
    for X, y in loader:
        y_pred = model(X)
        loss = linear_loss(y_pred, y)
        total_loss += loss.item() * X.shape[0]
    return total_loss / len(loader.dataset)

In [ ]:
# train_size = 10
# test_size = 200

# train the model
print ("train_loss", "test_loss", sep="\t")
for train_size in range(10, 201, 10):
# for train_size in range(1, 21):
#     for epsilon in range()
    test_size = max(1, train_size // 5)
    
    # want `y` to be either -1 or 1
    y_train = torch.randint(0, 2, (train_size,)) * 2.0 - 1
    x_train = torch.normal(mu*y_train, sigma)
    y_test = torch.randint(0, 2, (test_size,)) * 2.0 - 1
    x_test = torch.normal(mu*y_test, sigma)
    
#     batch_size = max(1, train_size // 5)

    # define datasets and data loaders
    train_set = Data.TensorDataset(x_train, y_train)
    train_loader = Data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
    test_set = Data.TensorDataset(x_test, y_test)
    test_loader = Data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

    
    for epoch in range(1, epochs+1):
        train_loss, test_loss = 0., 0.
        train_loss = epoch_adversarial(train_loader, model)
        test_loss = epoch_test(test_loader, model)
    # logging
#         if (epoch-1) % 10 == 0:
    print(*("{:.6f}".format(i) for i in (train_loss, test_loss)), sep="\t")

train_loss	test_loss
-0.383569	-5.597002
0.089163	-1.237262
-1.035395	-1.915035
-6.519569	-18.792042
0.468623	-4.046902
-0.092113	-3.951696
